In [1]:
import torch
print(torch.cuda.is_available())


True


In [3]:
import numpy as np

# Load existing feature matrix
A = np.load("feature_matrix_A.npy")


In [2]:
import torch
import torchvision.transforms as T


In [3]:
model = torch.hub.load("facebookresearch/dinov2", "dinov2_vitl14", pretrained=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Using cache found in /home/nemo/.cache/torch/hub/facebookresearch_dinov2_main
/home/nemo/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/nemo/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/nemo/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


: 

: 

In [ ]:
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

def normalize_vector(vector):
    """
    Normalize a vector to unit norm.
    """
    norm = np.linalg.norm(vector)
    return vector / norm if norm != 0 else vector

def identify(y, A, class_dict, lambda_=0.001):
    """
    Identify the class of a probe vector y given the gallery matrix A and a class dictionary.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    A_torch = torch.tensor(A, dtype=torch.float32, device=device)
    y_torch = torch.tensor(y, dtype=torch.float32, device=device)
    
    At = A_torch.T
    AtA = At @ A_torch
    Aty = At @ y_torch

    n = A_torch.shape[1]
    I = torch.eye(n, device=device, dtype=torch.float32)
    regularized_matrix = AtA + lambda_ * I

    try:
        x0 = torch.linalg.solve(regularized_matrix, Aty)
    except RuntimeError as e:
        logging.error(f"CUDA error during identification: {e}")
        return None

    x0 = x0.cpu().numpy()

    best_class = None
    min_error = float('inf')
    for cls, indices in class_dict.items():
        A_class = A[:, indices]
        x0_class = x0[indices]
        reconstruction = A_class @ x0_class
        error = np.linalg.norm(y - reconstruction, 2) ** 2
        if error < min_error:
            min_error = error
            best_class = cls

    return best_class

def extract_features_combined(image_path, model, transform):
    """
    Extract a feature vector for the image at image_path using DINOv2.
    """
    try:
        pil_img = Image.open(image_path).convert("L")
        # Replicate grayscale channel to 3-channel
        pil_img = Image.merge("RGB", (pil_img, pil_img, pil_img))
    except Exception as e:
        logging.error(f"Error loading image (PIL): {image_path} : {e}")
        return None

    img_tensor = transform(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        dino_features = model(img_tensor).cpu().numpy().flatten()
    dino_features = normalize_vector(dino_features)
    return dino_features
